# Relatório 2

**Nome:** Thiago Lopes <br>
**Matrícula:** 20100358 <br>
**Turma:** T2

# Bibliotecas python utilizadas

In [314]:
import math
import matplotlib.pyplot as plt
import numpy as np
from prettytable import PrettyTable
import sympy

# Funçoes Auxiliares

In [315]:
def pivot(A, b, k):
    max_index = np.argmax(np.abs(A[k:, k])) + k
    
    if max_index != k:
        A[[k, max_index]] = A[[max_index, k]]
        b[[k, max_index]] = b[[max_index, k]]


In [316]:
def back_substitution(U, y):
    m = len(y)
    x = np.zeros(m)
    
    for i in range(m - 1, -1, -1):
        x[i] = (y[i] - np.dot(U[i, i + 1:], x[i + 1:])) / U[i, i]
        
    return x


In [317]:
def forward_substitution2(L, b, Pivot):
    m = len(b)
    y = np.zeros(m)
    
    for i in range(m):
        y[i] = b[Pivot[i] - 1] - np.dot(L[i, :i], y[:i])        
    return y

In [318]:
def forward_substitution(L, b):
    n = L.shape[0]
    y = np.zeros_like(b)
    
    for i in range(n):
        y[i] = (b[i] - np.dot(L[i, :i], y[:i])) / L[i, i]
    
    return y

# Métodos implementados (Gaussian Elimination, LU Decomposition):

In [319]:
def gauss_elimination(A, b, use_pivoting=False):
    n = len(b)
    A = A.astype(float)
    b = b.astype(float)

    #eliminacao com ou sem pivotamento
    for k in range(n-1):
        if use_pivoting:  
            pivot(A, b, k)

        for i in range(k+1, n):
            m = A[i][k] / A[k][k]
            A[i][k] = 0
            
            for j in range(k+1, n):
                A[i][j] -= m * A[k][j]
                
            b[i] -= m * b[k]

    return back_substitution(A, b)


In [320]:
def LU_decomposition(A, b):
    A = A.astype(np.float64)
    b = b.astype(np.float64)

    m, n = A.shape
    
    if m != n:
        raise ValueError("Matrix must be square")
    
    Pivot = np.arange(1, m + 1)
    PdU = 1.0
    Info = 0
    LU = A.copy()

    for j in range(n):
        pivot(LU, b, j)
        
        Pivot[j], Pivot[np.argmax(np.abs(LU[j:, j])) + j] = Pivot[np.argmax(np.abs(LU[j:, j])) + j], Pivot[j]

        if LU[j, j] == 0:
            if Info == 0:
                Info = j + 1
            continue

        PdU *= LU[j, j]

        if abs(LU[j, j]) != 0:
            r = 1 / LU[j, j]
            for i in range(j + 1, m):
                Mult = LU[i, j] * r
                LU[i, j] = Mult
                LU[i, j + 1:] -= Mult * LU[j, j + 1:]
        else:
            if Info == 0:
                Info = j + 1

    if Info != 0:
        raise ValueError(f"Matrix is singular at row {Info}")

    y = forward_substitution2(LU, b, Pivot)

    x = back_substitution(LU, y)

    return x, LU, Pivot, PdU, Info

In [321]:
def cholesky_decomposition(A, b):
    n = A.shape[0]
    L = np.zeros_like(A)
    Det = 1.0
    Info = 0
    
    for j in range(n):
        Soma = 0.0
        for k in range(j):
            Soma += L[j, k] * L[j, k]
        
        t = A[j, j] - Soma
        if t > 0:
            L[j, j] = np.sqrt(t)
            r = 1 / L[j, j]
            Det *= t
        else:
            Info = j + 1
            raise ValueError("Matrix is not positive definite")
        
        for i in range(j + 1, n):
            Soma = 0.0
            for k in range(j):
                Soma += L[i, k] * L[j, k]
            L[i, j] = (A[i, j] - Soma) * r
    
    # Solve Ly = b using forward substitution with pivoting
    y = forward_substitution(L, b)
    
    # Solve L^Tx = y using backward substitution
    x = back_substitution(L.T, y)
    
    return x, L, Det, Info

# Exercícios

## Exercício 1

In [322]:
# Gauss eliminatioon
A = np.array([
    [2, 3, -1],
    [4, 4, -3],
    [2, -3, 1]])

b = np.array([5, 3, -1])

solution = gauss_elimination(A, b, use_pivoting=True)
print(solution)

[1. 2. 3.]


In [326]:
# LU Decomposition
A = np.array([
    [3, 2, 4],
    [0, 0.333, 0.666],
    [0, 0, -8]])

b = np.array([1, 1.666, 0])

# -------------------------------

# A = np.array([
#     [1, 0, 0],
#     [0.333, 1, 0],
#     [1.333, 1, 1]])

# b = np.array([1, 2, 3])

# -------------------------------

# A = np.array([
#     [1, 5, 5], 
#     [6, 9, 22], 
#     [32, 5., 5]])

# b = np.array([1, 2, 7])

# -------------------------------

# A = np.array([[1, 2, 4, 3],
#               [2, 0, 2, 1],
#               [4, 1, 3, 2],
#               [1, 0, 1, 1]], dtype=float)

# b = np.array([16, 13, 27, 7], dtype=float)

solution, LU, Pivot, PdU, Info = LU_decomposition(A, b)

print("Solution:", solution)
print("LU matrix:\n", LU)
print("Pivot indices:", Pivot)
print("Product of the diagonal of U:", PdU)
print("Info:", Info) 

Solution: [-3.002002  5.003003 -0.      ]
LU matrix:
 [[ 3.     2.     4.   ]
 [ 0.     0.333  0.666]
 [ 0.     0.    -8.   ]]
Pivot indices: [1 2 3]
Product of the diagonal of U: -7.992000000000001
Info: 0


In [324]:
# # cholesky_decomposition

# A = np.array([[4, 2, -4],
#                [2, 10, 4],
#                [-4, 4, 9]], dtype=float)

# b = np.array([0, 6, 5], dtype=float)

# x, L, Det, Info = cholesky_decomposition(A, b)

# print("Solution x:", x)
# print("Lower triangular matrix L:\n", L)
# print("Determinant:", Det)
# print("Info:", Info)